# 識別率の高いボクセルのブロードマンエリア特定

----  
  
引数：各被験者のディレクトリまでのパス  
  
---  
  
入力：識別率一覧 ACCURACY[loo]_Voxtimseries30_SVM.csv / ボクセル-ブロードマンエリア対応表 Voxel_Brodmann_Table.csv  
   
---
  
出力：Accuracy_Brodmann_Table.csv 
  
---  
  
各ボクセルがどのブロードマンエリアであるかを特定，識別率が高い順に並べ替える．  

In [58]:
from os.path import join as pathjoin
from pprint import pprint
# from nifti import NiftiImage
import glob
import numpy as np
import pandas as pd
import sys
import pickle
# import dill
import csv

In [79]:
# args = sys.argv
# PATH = args[1]

#jupyter notebookのときはここで指定
PATH = '../State-2fe_MaskBrodmann/'

Subject = ['20181029rn', '20181029su', '20181029tm']

PATH_raw = []

# 各被験者ごとのRawDataディレクトリまでのパス
for subName in Subject:
    
    PATH_raw = PATH_raw + [PATH + subName + '/mb/RawData/']


In [81]:

for sub in PATH_raw:
    print(sub)
    
    # 識別率一覧を記録したcsvファイルへのパス
    PATH_ac = []

    for name in glob.glob(sub + 'ACMID*'):
        PATH_ac = PATH_ac + [name]

    # ボクセル-ブロードマンエリア対応表を記録したファイルへのパス
    PATH_tab = sub + 'VoxelBrodmannTable.csv'

    # 識別率一覧を読み込み

    ac = pd.DataFrame(index = [], columns = [])

    for file in PATH_ac:

        acmid = pd.read_csv(file, header = 0)
        ac = pd.concat([ac, acmid])

    ac.columns = ['BAvoxelNum', 'accuracy(%)']

    # ボクセル-ブロードマンエリア対応表を読み込み
    tab = pd.read_csv(PATH_tab, header = 0)

    # acとtabをマージすることで各ボクセルのブロードマンエリアを特定
    AcTab = pd.merge(ac, tab, on = ['BAvoxelNum'], how = 'left')

    # BAvoxelNumとaccuracyを行名に設定することで(MultiIndex)重複を分かりやすくする
    # (一つのボクセルに対して複数のブロードマンエリアが対応していることがある）
    AcTab = AcTab.set_index(['BAvoxelNum', 'accuracy(%)'])

    # accuracyでソートすることで識別率の高い順に並べる
    # (MultiIndexの時は)levelでどのインデックスでソートするか指定，ascending = Falseとすることで昇順に
    AcTab = AcTab.sort_index(level = 'accuracy(%)', ascending = False)
    
    PATH_csv = sub + 'AcVoxelBrodmannTable.csv'
    print(PATH_csv)
    AcTab.to_csv(PATH_csv)

../State-2fe_MaskBrodmann/20181029rn/mb/RawData/
../State-2fe_MaskBrodmann/20181029rn/mb/RawData/AcVoxelBrodmannTable.csv
../State-2fe_MaskBrodmann/20181029su/mb/RawData/
../State-2fe_MaskBrodmann/20181029su/mb/RawData/AcVoxelBrodmannTable.csv
../State-2fe_MaskBrodmann/20181029tm/mb/RawData/
../State-2fe_MaskBrodmann/20181029tm/mb/RawData/AcVoxelBrodmannTable.csv


In [ ]:
ri